In [1]:
import numpy as np
import pandas as pd
import time
import timeit
from datetime import datetime
import os
import glob
import natsort
import sys
import cv2
from PIL import Image
import random
import copy
import sys
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
from sklearn.model_selection import train_test_split, KFold
from monai.losses import TverskyLoss as TverskyLoss

from metrics import *
from data_loader import *
from utils import *
from train import *

model_name = 'ReflexNet'
exec(f'from {model_name} import *')

Dataset_dir = '../Total_Datasets/Reflex_Dataset'

iterations = [1, 30]
train_size=0.6

in_channels = 3
number_of_classes=1
epochs = 50
batch_size = 8

devices = [0,1]

optimizer = 'AdamW'
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}

lr_scheduler = 'CosineAnnealingLR'
T_max = epochs
T_0 = 50
eta_min = 1e-6
lr_scheduler_args = {'lr_scheduler': lr_scheduler, 'T_max': T_max, 'T_0': T_0, 'eta_min': eta_min}

loss_function = 'Tversky Focal Loss'
reduction = 'mean'
gamma = 2.0
weight = None
loss_function_args = {'loss_function': loss_function, 'reduction': reduction, 'gamma': gamma, 'weight': weight}

In [2]:
def contduct_experiments(iteration, model_name, model, train_loader, validation_loader, test_loader, Optimizer, lr,  number_of_classes, epochs, Metrics,df,device, transform):
    start = timeit.default_timer()
    train_bool=True
    test_bool=True
    if loss_function == 'Tversky Focal Loss':
        criterion=TverskyLoss()
    if Optimizer=='Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    elif Optimizer == 'SGD':
        momentum = 0.9
        weight_decay = 1e-4
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum ,weight_decay=weight_decay)
    elif Optimizer =='AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    if lr_scheduler_args['lr_scheduler'] == 'CosineAnnealingLR':
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = lr_scheduler_args['T_max'], eta_min = lr_scheduler_args['eta_min'])
    
    os.makedirs(output_dir, exist_ok = True)
    control_random_seed(seed)
    if train_bool:
        now = datetime.now()
        Train_date=now.strftime("%y%m%d_%H%M%S")
        print('Training Start Time:',Train_date)
        best=9999
        best_epoch=1
        Early_Stop=0
        loss_saver = LossSaver()
        train_start_time = timeit.default_timer()
        for epoch in range(1, epochs+1):
            Train_Loss = train(train_loader, epoch, 
              model, criterion, optimizer, device
              )
            lr_scheduler.step()
            outputs, targets  \
            = validate(validation_loader, 
              model, criterion, device
              )
            Val_Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)            
            iou = np.round(Intersection_over_Union(outputs, targets),3)
            dice = np.round(Dice_Coefficient(outputs, targets),3)
            now = datetime.now()
            date=now.strftime("%y%m%d_%H%M%S")
            print(str(epoch)+'EP('+date+'):',end=' ')
            print('T_Loss: ' + str(Train_Loss), end=' ')
            print('V_Loss: ' + str(Val_Loss), end=' ')
            print('IoU: ' + str(iou), end=' ')
            print('Dice: ' + str(dice), end=' ')
            
            loss_saver.update(Train_Loss, Val_Loss)
            loss_saver.save_as_csv(f'{output_dir}/Losses_{Experiments_Time}.csv')
            if Val_Loss<best:
                Early_Stop = 0
                torch.save(model.state_dict(), f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt')
                best_epoch = epoch
                best = Val_Loss
                print('Best Epoch:',best_epoch,'Loss:',Val_Loss)
            else:
                print('')
                Early_Stop+=1
            if Early_Stop>=15:
                break
        train_stop_time = timeit.default_timer()
    if test_bool:
        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Test Start Time:',date)
        outputs, targets, image_paths \
            = validate(test_loader, 
              model, criterion, device,
            model_path=f'{output_dir}/{Train_date}_{model_name}_Iter_{iteration}.pt',
                       return_image_paths=True
              )        
        Loss = np.round(criterion(outputs,targets).cpu().numpy(),6)
        iou = np.round(Intersection_over_Union(outputs, targets),3)
        dice = np.round(Dice_Coefficient(outputs, targets),3)
        recall, precision, f1 = Confusion_Matrix(outputs, targets) 
        recall = np.round(recall.cpu().numpy()[0],3); precision = np.round(precision.cpu().numpy()[0],3); f1 = np.round(f1.cpu().numpy()[0],3);
                
        now = datetime.now()
        date=now.strftime("%y%m%d_%H%M%S")
        print('Best Epoch:',best_epoch)
        print('Test('+date+'): '+'Loss: ' + str(Loss),end=' ')
        print('IoU: ' + str(iou), end=' ')
        print('Dice: ' + str(dice), end=' ')
        print('Recall: ' + str(recall), end=' ')
        print('Precision: ' + str(precision), end=' ')
        print('F1 Score: ' + str(f1), end='\n')
                            
        stop = timeit.default_timer();m, s = divmod((train_stop_time - train_start_time)/epoch, 60);h, m = divmod(m, 60);Time_per_Epoch = "%02d:%02d:%02d" % (h, m, s);
        m, s = divmod(stop - start, 60);h, m = divmod(m, 60);Time = "%02d:%02d:%02d" % (h, m, s);
        total_params = sum(p.numel() for p in model.parameters()); total_params = format(total_params , ',');
        Performances = [Experiments_Time, Train_date, iteration, model_name, best, Loss, iou, dice, recall, precision, f1, total_params,Time, best_epoch, Time_per_Epoch, loss_function, lr, batch_size, epochs]
        df = df.append(pd.Series(Performances, index=df.columns), ignore_index=True)
        os.makedirs(f'{output_dir}/test_outputs', exist_ok = True)
        outputs = outputs.cpu().numpy()
        for output, image_path in zip(outputs, image_paths):
            np.save(f'{output_dir}/test_outputs/{os.path.basename(image_path)}', output)
    now = datetime.now()
    date=now.strftime("%y%m%d_%H%M%S")
    print('End',date)
    return df
def str_to_class(classname):
    return getattr(sys.modules[__name__], classname)

In [3]:
now = datetime.now()
Experiments_Time=now.strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)
Metrics=['Experiment Time','Train Time', 'Iteration','Model Name', 'Val_Loss', 'Test_Loss', 'IoU', 'Dice', 'Recall', 'Precision', 'F1 Score', 'Total Params','Train-Predction Time','Best Epoch','Time per Epoch', 'Loss Function', 'LR', 'Batch size', '#Epochs']
df = pd.DataFrame(index=None, columns=Metrics)
output_root = f'output/output_{Experiments_Time}'
os.makedirs(output_root, exist_ok = True)
    
for iteration in range(iterations[0], iterations[1]+1):
    seed = iteration
    image_path_list=natsort.natsorted(glob.glob(Dataset_dir+'/Originals/*')); target_path_list=[]
    for image_path in image_path_list:
        target_path_list.append(Dataset_dir+'/Masks/'+os.path.basename(image_path))
    num_dataset = len(target_path_list)
    indices = list(range(num_dataset))
    split1=int(train_size*num_dataset)
    split2=int((train_size+(1-train_size)/2)*num_dataset)
    control_random_seed(seed)
    np.random.shuffle(indices)
    train_idx, validation_idx, test_idx = indices[:split1], indices[split1:split2], indices[split2:]
    train_image_path_list=[]
    train_target_path_list=[]
    validation_image_path_list=[]
    validation_target_path_list=[]
    test_image_path_list=[]
    test_target_path_list=[]
    for i, index in enumerate(indices):
        if i<split1:
            train_image_path_list.append(image_path_list[index])
            train_target_path_list.append(target_path_list[index])
        elif split1<=i and i<split2:
            validation_image_path_list.append(image_path_list[index])
            validation_target_path_list.append(target_path_list[index])
        else:
            test_image_path_list.append(image_path_list[index])
            test_target_path_list.append(target_path_list[index])
    train_dataset = ImagesDataset(train_image_path_list, train_target_path_list, aug=True)
    validation_dataset = ImagesDataset(validation_image_path_list, validation_target_path_list, aug=False)
    test_dataset = ImagesDataset(test_image_path_list, test_target_path_list, aug=False)
    train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size,
    num_workers=4, pin_memory=True, shuffle=True, 
    )
    validation_loader = torch.utils.data.DataLoader(
        validation_dataset, batch_size=batch_size, 
        num_workers=4, pin_memory=True,
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, 
        num_workers=4, pin_memory=True,
    )
    print(f'{model_name} (Iter {iteration})')
    output_dir = output_root + f'/{model_name}_Iter_{iteration}'
    copy_sourcefile(output_dir, src_dir='src')
    control_random_seed(seed)
    model=str_to_class(model_name)(in_channels, number_of_classes)
    device = torch.device("cuda:"+str(devices[0]))
    if len(devices)>1:
        model = torch.nn.DataParallel(model, device_ids = devices ).to(device)
    else:
        model = model.to(device)
    df = contduct_experiments(seed, model_name, model, train_loader, validation_loader, test_loader,  optimizer, lr,  number_of_classes, epochs, Metrics, df, device,None)
    df.to_csv(output_root+'/'+'Reflex_Seg_'+Experiments_Time+'.csv', index=False, header=True, encoding="cp949")


Experiment Start Time: 240121_032349
ReflexNet (Iter 1)
Training Start Time: 240121_032350
1EP(240121_032417): T_Loss: 0.839045 V_Loss: 0.999631 IoU: 0.0 Dice: 0.0 Best Epoch: 1 Loss: 0.999631
Test Start Time: 240121_032417
Best Epoch: 1
Test(240121_032419): Loss: 0.999592 IoU: 0.0 Dice: 0.0 Recall: 0.0 Precision: 0.0 F1 Score: 0.0
End 240121_032419
